In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('new_data.csv')
x = dataset.iloc[:, :-2].values  
y = dataset.iloc[:, -2:].values


In [3]:
print(x)
print(x.shape)

[['103.216.15.12' '84.9.164.252' 31225 ... 503 'HTTP'
  'Jamshedpur, Sikkim']
 ['78.199.217.198' '66.191.137.154' 17245 ... 1174 'HTTP'
  'Bilaspur, Nagaland']
 ['63.79.210.48' '198.219.82.17' 16811 ... 306 'HTTP' 'Bokaro, Rajasthan']
 ...
 ['162.35.217.57' '98.107.0.15' 22505 ... 1043 'DNS'
  'Mahbubnagar, Himachal Pradesh']
 ['208.72.233.205' '173.79.112.252' 20013 ... 483 'FTP'
  'Rourkela, Arunachal Pradesh']
 ['14.102.21.108' '109.198.45.7' 50137 ... 1175 'HTTP'
  'Pudukkottai, West Bengal']]
(40000, 8)


In [4]:
print(y)
print(y.shape)

[['Low' 'Logged']
 ['Low' 'Blocked']
 ['Low' 'Ignored']
 ...
 ['Low' 'Blocked']
 ['Low' 'Ignored']
 ['Medium' 'Logged']]
(40000, 2)


encoding data

In [5]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder


columns_to_encode = [0, 1, 4, 6, 7]

column_transformer = ColumnTransformer(
    transformers=[
        ('categorical', OrdinalEncoder(), columns_to_encode), 
        ('numeric', 'passthrough', [i for i in range(x.shape[1]) if i not in columns_to_encode])  
    ]
)

x[:] = column_transformer.fit_transform(x)


In [6]:


columns_to_encode_y = [0, 1]
ordinal_encoder_y = OrdinalEncoder()
y[:] = ordinal_encoder_y.fit_transform(y)




In [58]:
print(y)

[[1.0 2.0]
 [1.0 0.0]
 [1.0 1.0]
 ...
 [1.0 0.0]
 [1.0 1.0]
 [2.0 2.0]]


In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

In [23]:
print(x_train.shape)
print(y_train.shape)

(30000, 8)
(30000, 2)


FEATURE SCALING

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(X=x_train)
x_test = sc.transform(x_test)

In [25]:
print(x_train.shape)
print(y_train.shape)

(30000, 8)
(30000, 2)


MODEL TRAINING

In [27]:
y_train = y_train.astype(int)


In [28]:
from sklearn.ensemble import RandomForestClassifier


classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)


classifier.fit(x_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [66]:
new_input = [
    '4.255.187.165',
    '136.159.186.239',
    39934,
    5259,
    'TCP',
    838,
    'HTTP',
    'Solapur, Telangana'
]

encoded_input = column_transformer.transform([new_input])
scaled_input = sc.transform(encoded_input)
prediction = classifier.predict(scaled_input)
print(prediction)


[[0 1]]


In [44]:
y_pred = classifier.predict(x_test)
arr=np.column_stack((y_pred, y_test))
print(arr.shape)

(10000, 4)


In [46]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_test_flat = y_test.reshape(-1)
y_pred_flat = y_pred.reshape(-1)
y_test_flat = y_test_flat.astype(int)
y_pred_flat = y_pred_flat.astype(int)
cm = confusion_matrix(y_test_flat, y_pred_flat)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[2550 2255 1943]
 [2576 2169 1837]
 [2602 2213 1855]]


In [49]:
accuracy = accuracy_score(y_test_flat, y_pred_flat)
print(accuracy)

0.3287
